# Vision Transformer 农作物制图训练和评估

本notebook演示如何使用Vision Transformer模型进行多光谱时序数据的农作物分类任务。

## 🔄 双模式设计

**📋 本notebook支持两种运行模式：**

### 🧪 TEST模式 
- **目的**: 本地快速测试代码正确性
- **配置**: 轻量级模型，小数据集，少量epochs
- **用时**: ~3分钟
- **适用**: 开发调试，验证代码逻辑
- **环境**: 本地CPU/GPU，低内存要求

### 🚀 TRAIN模式
- **目的**: 云端GPU完整训练获得最佳性能
- **配置**: 标准模型，完整数据集，完整epochs
- **用时**: ~8-16小时
- **适用**: 正式训练，获得部署模型
- **环境**: 云端GPU，高性能要求

**💡 使用建议：**
1. 本地开发时使用TEST模式验证代码
2. 确认无误后切换到TRAIN模式进行完整训练
3. 在云端GPU上运行TRAIN模式以获得最佳效果

---

## 目录
1. [运行模式配置](#运行模式配置)
2. [环境设置和导入](#环境设置和导入)
3. [数据加载和探索](#数据加载和探索)
4. [模型创建和配置](#模型创建和配置)
5. [训练过程](#训练过程)
6. [模型评估](#模型评估)
7. [结果可视化](#结果可视化)
8. [注意力机制分析](#注意力机制分析)
9. [模型推理](#模型推理)"

In [ ]:
# 🔧 运行模式配置
# 设置 RUNNING_MODE 来选择运行模式
# - "TEST": 本地测试模式，快速验证代码正确性
# - "TRAIN": 云端训练模式，完整训练流程

RUNNING_MODE = "TEST"  # 👈 修改这里来切换模式: "TEST" 或 "TRAIN"

print(f"🔄 当前运行模式: {RUNNING_MODE}")

if RUNNING_MODE == "TEST":
    print("🧪 TEST模式 - 用于本地测试:")
    print("  ✓ 使用小数据集")
    print("  ✓ 快速训练 (5 epochs)")
    print("  ✓ 小模型配置")
    print("  ✓ 快速验证代码正确性")
elif RUNNING_MODE == "TRAIN":
    print("🚀 TRAIN模式 - 用于云端GPU训练:")
    print("  ✓ 使用完整数据集")
    print("  ✓ 完整训练 (100 epochs)")
    print("  ✓ 标准模型配置")
    print("  ✓ 最佳性能优化")
else:
    raise ValueError("RUNNING_MODE 必须是 'TEST' 或 'TRAIN'")

## 2. 环境设置和导入

In [1]:
# 导入必要的库
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 设置matplotlib中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU设备: {torch.cuda.get_device_name(0)}")
    print(f"GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

PyTorch版本: 2.7.1
CUDA可用: False


## 3. 数据加载和探索

In [ ]:
# 📋 根据运行模式配置参数
if RUNNING_MODE == "TEST":
    # 🧪 TEST模式配置 - 快速测试代码正确性
    config = {
        # 数据参数 - 使用更小的数据量
        'data_path': '../dataset',
        'patch_size': 32,     # 更小的patch大小
        'stride': 16,         # 更小的stride，减少数据量
        'test_size': 0.3,     # 更大的测试集比例
        'val_size': 0.2,      # 更大的验证集比例
        'batch_size': 2,      # 小batch size用于测试
        'num_workers': 2,     # 少量工作进程
        
        # 模型参数 - 轻量级配置
        'input_channels': 8,
        'temporal_steps': 28,
        'model_patch_size': 8,
        'embed_dim': 128,     # 更小的嵌入维度
        'num_layers': 3,      # 更少的层数
        'num_heads': 4,       # 更少的注意力头
        'mlp_ratio': 2.0,     # 更小的MLP比例
        'dropout': 0.0,       # 简化配置
        
        # 训练参数 - 快速训练
        'epochs': 5,          # 很少的epochs用于测试
        'learning_rate': 1e-3, # 稍大的学习率快速收敛
        'weight_decay': 0.01,
        'gradient_accumulation_steps': 1,
        'max_grad_norm': 1.0,
        
        # 损失函数
        'use_combined_loss': False,  # 简单损失函数
        'focal_gamma': 2.0,
        'label_smoothing': 0.05,
        
        # 学习率调度
        'scheduler_type': 'none',  # 不使用调度器
        'min_lr': 1e-5,
        
        # 其他
        'patience': 5,        # 短耐心值
        'save_dir': './checkpoints_test',
        'augment_train': False  # 不使用数据增强
    }
    print("🧪 使用TEST模式配置 - 适合快速验证代码")
    
elif RUNNING_MODE == "TRAIN":
    # 🚀 TRAIN模式配置 - 完整训练流程
    config = {
        # 数据参数 - 使用完整数据集
        'data_path': '../dataset',
        'patch_size': 64,     # 标准patch大小
        'stride': 32,         # 标准stride
        'test_size': 0.2,
        'val_size': 0.1,
        'batch_size': 8,      # 适中的batch size
        'num_workers': 4,
        
        # 模型参数 - 标准配置
        'input_channels': 8,
        'temporal_steps': 28,
        'model_patch_size': 8,
        'embed_dim': 256,     # 标准嵌入维度
        'num_layers': 6,      # 标准层数
        'num_heads': 8,       # 标准注意力头数
        'mlp_ratio': 4.0,     # 标准MLP比例
        'dropout': 0.1,
        
        # 训练参数 - 完整训练
        'epochs': 100,        # 完整训练轮数
        'learning_rate': 1e-4, # 标准学习率
        'weight_decay': 1e-4,
        'gradient_accumulation_steps': 2,
        'max_grad_norm': 1.0,
        
        # 损失函数
        'use_combined_loss': True,  # 使用组合损失
        'focal_gamma': 2.5,
        'label_smoothing': 0.1,
        
        # 学习率调度
        'scheduler_type': 'cosine',
        'min_lr': 1e-6,
        
        # 其他
        'patience': 15,       # 长耐心值
        'save_dir': './checkpoints_train',
        'augment_train': True  # 使用数据增强
    }
    print("🚀 使用TRAIN模式配置 - 适合完整训练")

# 创建保存目录
Path(config['save_dir']).mkdir(parents=True, exist_ok=True)

print(f"\n📋 当前配置 ({RUNNING_MODE}模式):")
for key, value in config.items():
    print(f"  {key}: {value}")

if RUNNING_MODE == "TEST":
    print("\n⚠️ 注意: TEST模式使用简化配置，仅用于验证代码正确性!")
else:
    print("\n💪 TRAIN模式: 使用完整配置进行最佳性能训练!")

In [ ]:
# 🔄 加载和准备数据（适配不同模式）
print(f"🔄 加载数据 ({RUNNING_MODE}模式)...")

train_loader, val_loader, test_loader, data_info = prepare_data(
    data_path=config['data_path'],
    patch_size=config['patch_size'],
    stride=config['stride'],
    test_size=config['test_size'],
    val_size=config['val_size'],
    batch_size=config['batch_size'],
    num_workers=config['num_workers'],
    augment_train=config['augment_train']
)

print(f"\n📊 数据集信息 ({RUNNING_MODE}模式):")
print(f"  类别数量: {data_info['num_classes']}")
print(f"  训练批次: {len(train_loader)}")
print(f"  验证批次: {len(val_loader)}")
print(f"  测试批次: {len(test_loader)}")
print(f"  输入形状: {data_info['input_shape']}")

# 估算数据量
total_train_samples = len(train_loader) * config['batch_size']
total_val_samples = len(val_loader) * config['batch_size']
print(f"  训练样本数: ~{total_train_samples}")
print(f"  验证样本数: ~{total_val_samples}")

if RUNNING_MODE == "TEST":
    print(f"\n🧪 TEST模式数据量: 使用小数据集快速测试")
    print(f"  预计单epoch时间: ~30秒")
    print(f"  总训练时间: ~3分钟")
else:
    print(f"\n🚀 TRAIN模式数据量: 使用完整数据集")
    print(f"  预计单epoch时间: ~5-10分钟")
    print(f"  总训练时间: ~8-16小时")

# 保存数据信息
save_data_info(data_info, f"{config['save_dir']}/data_info.pkl")

# 显示类别信息
print(f"\n🏷️ 类别信息:")
for idx, name in data_info['class_names'].items():
    print(f"  {idx}: {name}")

print(f"\n⚖️ 类别权重: {data_info['class_weights'].numpy()}")

In [ ]:
# 📋 根据运行模式配置参数
if RUNNING_MODE == "TEST":
    # 🧪 TEST模式配置 - 快速测试代码正确性
    config = {
        # 数据参数 - 使用更小的数据量
        'data_path': '../dataset',
        'patch_size': 32,     # 更小的patch大小
        'stride': 16,         # 更小的stride，减少数据量
        'test_size': 0.3,     # 更大的测试集比例
        'val_size': 0.2,      # 更大的验证集比例
        'batch_size': 2,      # 小batch size用于测试
        'num_workers': 2,     # 少量工作进程
        
        # 模型参数 - 轻量级配置
        'input_channels': 8,
        'temporal_steps': 28,
        'model_patch_size': 8,
        'embed_dim': 128,     # 更小的嵌入维度
        'num_layers': 3,      # 更少的层数
        'num_heads': 4,       # 更少的注意力头
        'mlp_ratio': 2.0,     # 更小的MLP比例
        'dropout': 0.0,       # 简化配置
        
        # 训练参数 - 快速训练
        'epochs': 5,          # 很少的epochs用于测试
        'learning_rate': 1e-3, # 稍大的学习率快速收敛
        'weight_decay': 0.01,
        'gradient_accumulation_steps': 1,
        'max_grad_norm': 1.0,
        
        # 损失函数
        'use_combined_loss': False,  # 简单损失函数
        'focal_gamma': 2.0,
        'label_smoothing': 0.05,
        
        # 学习率调度
        'scheduler_type': 'none',  # 不使用调度器
        'min_lr': 1e-5,
        
        # 其他
        'patience': 5,        # 短耐心值
        'save_dir': '../Models/vision-transformer/test',  # 修改输出路径
        'augment_train': False  # 不使用数据增强
    }
    print("🧪 使用TEST模式配置 - 适合快速验证代码")
    
elif RUNNING_MODE == "TRAIN":
    # 🚀 TRAIN模式配置 - 完整训练流程
    config = {
        # 数据参数 - 使用完整数据集
        'data_path': '../dataset',
        'patch_size': 64,     # 标准patch大小
        'stride': 32,         # 标准stride
        'test_size': 0.2,
        'val_size': 0.1,
        'batch_size': 8,      # 适中的batch size
        'num_workers': 4,
        
        # 模型参数 - 标准配置
        'input_channels': 8,
        'temporal_steps': 28,
        'model_patch_size': 8,
        'embed_dim': 256,     # 标准嵌入维度
        'num_layers': 6,      # 标准层数
        'num_heads': 8,       # 标准注意力头数
        'mlp_ratio': 4.0,     # 标准MLP比例
        'dropout': 0.1,
        
        # 训练参数 - 完整训练
        'epochs': 100,        # 完整训练轮数
        'learning_rate': 1e-4, # 标准学习率
        'weight_decay': 1e-4,
        'gradient_accumulation_steps': 2,
        'max_grad_norm': 1.0,
        
        # 损失函数
        'use_combined_loss': True,  # 使用组合损失
        'focal_gamma': 2.5,
        'label_smoothing': 0.1,
        
        # 学习率调度
        'scheduler_type': 'cosine',
        'min_lr': 1e-6,
        
        # 其他
        'patience': 15,       # 长耐心值
        'save_dir': '../Models/vision-transformer/train',  # 修改输出路径
        'augment_train': True  # 使用数据增强
    }
    print("🚀 使用TRAIN模式配置 - 适合完整训练")

# 创建保存目录
Path(config['save_dir']).mkdir(parents=True, exist_ok=True)

print(f"\n📋 当前配置 ({RUNNING_MODE}模式):")
for key, value in config.items():
    print(f"  {key}: {value}")

if RUNNING_MODE == "TEST":
    print("\n⚠️ 注意: TEST模式使用简化配置，仅用于验证代码正确性!")
else:
    print("\n💪 TRAIN模式: 使用完整配置进行最佳性能训练!")

print(f"\n💾 模型输出目录: {config['save_dir']}")

## 4. 模型创建和配置

## 3. 模型创建和配置

In [ ]:
# 设置设备
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(f"🖥️ 使用设备: {device}")

# 创建Transformer模型
model = create_transformer_model(
    input_channels=config['input_channels'],
    temporal_steps=config['temporal_steps'],
    num_classes=data_info['num_classes'],
    patch_size=config['model_patch_size'],
    embed_dim=config['embed_dim'],
    num_layers=config['num_layers'],
    num_heads=config['num_heads'],
    mlp_ratio=config['mlp_ratio'],
    dropout=config['dropout']
).to(device)

# 计算参数量
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n🔧 模型信息:")
print(f"  总参数量: {total_params:,}")
print(f"  可训练参数: {trainable_params:,}")
print(f"  模型大小: {total_params * 4 / (1024**2):.1f} MB (FP32)")

# 测试前向传播
model.eval()
with torch.no_grad():
    test_input = sample_batch_x[:2].to(device)
    test_output = model(test_input)
    print(f"\n🧪 前向传播测试:")
    print(f"  输入形状: {test_input.shape}")
    print(f"  输出形状: {test_output.shape}")
    print(f"  输出数值范围: [{test_output.min():.3f}, {test_output.max():.3f}]")

print("\n✅ 模型创建成功!")

## 5. 训练过程

In [ ]:
# 🚀 训练准备（适配不同模式）
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_miou': [],
    'learning_rates': []
}

best_miou = 0.0
start_time = time.time()

if RUNNING_MODE == "TEST":
    print(f"🧪 开始TEST模式训练 - Vision Transformer ({config['epochs']} epochs)")
    print(f"⚡ 目标: 快速验证代码正确性")
    print(f"🔧 模型配置: 轻量级 (embed_dim={config['embed_dim']}, layers={config['num_layers']})")
else:
    print(f"🚀 开始TRAIN模式训练 - Vision Transformer ({config['epochs']} epochs)")
    print(f"🎯 目标: 获得最佳模型性能")
    print(f"💪 模型配置: 标准配置 (embed_dim={config['embed_dim']}, layers={config['num_layers']})")

print(f"⏰ 开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 特征: {config['num_layers']}层深度, {config['embed_dim']}嵌入维度, {config['num_heads']}注意力头")
print("=" * 80)

In [ ]:
# 训练历史记录
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_miou': [],
    'learning_rates': []
}

best_miou = 0.0
start_time = time.time()

print(f"🚀 开始训练 Transformer 模型 ({config['epochs']} epochs)")
print(f"⏰ 开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

In [ ]:
# 训练循环
for epoch in range(1, config['epochs'] + 1):
    epoch_start_time = time.time()
    
    print(f"\nEpoch {epoch}/{config['epochs']}")
    print("-" * 50)
    
    # 训练阶段
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device, scaler, epoch,
        scheduler=None,  # 我们手动调用scheduler
        scheduler_step_per_batch=False,
        gradient_accumulation_steps=config['gradient_accumulation_steps'],
        max_grad_norm=config['max_grad_norm']
    )
    
    # 验证阶段
    val_loss, val_acc, val_metrics = validate(
        model, val_loader, criterion, device, data_info['num_classes']
    )
    
    # 学习率调度
    if scheduler is not None:
        scheduler.step()
    
    # 获取当前学习率
    current_lr = optimizer.param_groups[0]['lr']
    
    # 记录历史
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_miou'].append(val_metrics['mean_iou'])
    history['learning_rates'].append(current_lr)
    
    # 计算epoch时间
    epoch_time = time.time() - epoch_start_time
    
    # 打印结果
    print(f"Epoch {epoch} 结果:")
    print(f"  训练 - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
    print(f"  验证 - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")
    print(f"  验证 mIoU: {val_metrics['mean_iou']:.4f}")
    print(f"  学习率: {current_lr:.2e}")
    print(f"  耗时: {epoch_time:.1f}s")
    
    # 保存最佳模型
    if val_metrics['mean_iou'] > best_miou:
        best_miou = val_metrics['mean_iou']
        save_checkpoint({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'scaler_state_dict': scaler.state_dict(),
            'best_miou': best_miou,
            'config': config,
            'metrics': val_metrics
        }, f"{config['save_dir']}/best_model.pth")
        print(f"  💾 新的最佳模型已保存! (mIoU: {best_miou:.4f})")
    
    # 早停检查
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("\n⏹️ 触发早停!")
        break
    
    # 每5个epoch显示一次进度
    if epoch % 5 == 0:
        total_time = time.time() - start_time
        avg_epoch_time = total_time / epoch
        estimated_total = avg_epoch_time * config['epochs']
        print(f"  📊 进度: {epoch/config['epochs']*100:.1f}%, 预计剩余时间: {(estimated_total - total_time)/60:.1f}分钟")

total_training_time = time.time() - start_time
print(f"\n🎉 训练完成!")
print(f"⏱️ 总训练时间: {total_training_time/60:.1f}分钟")
print(f"🏆 最佳 mIoU: {best_miou:.4f}")

## 6. 模型评估

## 5. 模型评估

In [ ]:
# 加载最佳模型进行评估
print("📂 加载最佳模型进行评估...")

# 重新创建模型
eval_model = create_transformer_model(
    input_channels=config['input_channels'],
    temporal_steps=config['temporal_steps'],
    num_classes=data_info['num_classes'],
    patch_size=config['model_patch_size'],
    embed_dim=config['embed_dim'],
    num_layers=config['num_layers'],
    num_heads=config['num_heads'],
    mlp_ratio=config['mlp_ratio'],
    dropout=config['dropout']
).to(device)

# 加载最佳权重
checkpoint = load_checkpoint(f"{config['save_dir']}/best_model.pth", eval_model)
eval_model.eval()

print(f"✅ 模型加载成功 (训练epoch: {checkpoint['epoch']})")

In [ ]:
# 在测试集上评估
print("🧪 在测试集上评估模型...")

eval_model.eval()
all_preds = []
all_targets = []
all_probs = []

with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(device)
        targets_cpu = targets.numpy()
        
        outputs = eval_model(data)  # (batch, height, width, num_classes)
        probs = torch.softmax(outputs, dim=-1)
        _, predicted = outputs.max(-1)
        
        all_preds.append(predicted.cpu().numpy())
        all_targets.append(targets_cpu)
        all_probs.append(probs.cpu().numpy())

# 合并所有批次
all_preds = np.concatenate(all_preds).flatten()
all_targets = np.concatenate(all_targets).flatten()

# 计算详细指标
metrics = calculate_metrics(all_preds, all_targets, data_info['num_classes'])
metrics['class_names'] = data_info['class_names']

print(f"\n📊 测试集评估结果:")
print(f"  总体准确率: {metrics['overall_accuracy']:.4f}")
print(f"  平均准确率: {metrics['mean_accuracy']:.4f}")
print(f"  平均IoU: {metrics['mean_iou']:.4f}")
print(f"  平均精确率: {metrics['mean_precision']:.4f}")
print(f"  平均召回率: {metrics['mean_recall']:.4f}")

## 7. 结果可视化

## 6. 结果可视化

In [ ]:
# 绘制混淆矩阵
plot_confusion_matrix(
    metrics['confusion_matrix'],
    list(data_info['class_names'].values()),
    save_path=f"{config['save_dir']}/confusion_matrix.png",
    title="Transformer Confusion Matrix"
)

print("📊 混淆矩阵已保存")

In [ ]:
# 绘制类别性能图表
plot_class_performance(
    metrics, 
    save_path=f"{config['save_dir']}/class_performance.png",
    title="Transformer Per-Class Performance"
)

print("📈 类别性能图表已保存")

## 8. 注意力机制分析

## 7. 模型推理

In [ ]:
# 演示单个patch的推理
print("🔮 演示模型推理...")

# 获取一个样本
sample_x = test_batch_x[0:1].to(device)  # 取第一个样本
sample_y = test_batch_y[0:1]

# 推理
eval_model.eval()
with torch.no_grad():
    # 前向传播
    inference_start = time.time()
    outputs = eval_model(sample_x)
    inference_time = time.time() - inference_start
    
    # 获取预测和置信度
    probs = torch.softmax(outputs, dim=-1)
    confidence, predictions = probs.max(-1)
    
    # 计算准确率
    accuracy = (predictions.cpu() == sample_y).float().mean().item()

print(f"\n⚡ 推理性能:")
print(f"  推理时间: {inference_time*1000:.2f}ms")
print(f"  输入形状: {sample_x.shape}")
print(f"  输出形状: {outputs.shape}")
print(f"  预测准确率: {accuracy:.4f}")
print(f"  平均置信度: {confidence.mean():.4f}")
print(f"  最小置信度: {confidence.min():.4f}")
print(f"  最大置信度: {confidence.max():.4f}")

## 9. 模型推理

In [ ]:
print("=" * 80)
print(f"🎯 VISION TRANSFORMER 农作物制图模型训练总结 ({RUNNING_MODE}模式)")
print("=" * 80)

print(f"\n🔄 运行模式: {RUNNING_MODE}")
if RUNNING_MODE == "TEST":
    print("  ✓ 快速验证代码正确性")
    print("  ✓ 使用轻量级模型配置")
    print("  ✓ 少量训练轮数")
    print("  ✓ 适合本地开发环境")
else:
    print("  ✓ 完整训练流程")
    print("  ✓ 标准模型配置")
    print("  ✓ 充足训练轮数")
    print("  ✓ 适合云端GPU环境")

print(f"\n📊 模型配置:")
print(f"  模型类型: Vision Transformer")
print(f"  嵌入维度: {config['embed_dim']}")
print(f"  层数: {config['num_layers']}")
print(f"  注意力头数: {config['num_heads']}")
print(f"  Patch大小: {config['model_patch_size']}x{config['model_patch_size']}")
print(f"  总参数量: {total_params:,}")

print(f"\n🏃 训练过程:")
print(f"  训练epochs: {len(history['train_loss'])}")
print(f"  总训练时间: {total_training_time/60:.1f}分钟")
print(f"  平均每epoch: {total_training_time/len(history['train_loss']):.1f}秒")
print(f"  最佳验证mIoU: {best_miou:.4f}")
if len(history['learning_rates']) > 0:
    print(f"  最终学习率: {history['learning_rates'][-1]:.2e}")

print(f"\n📈 最终性能:")
print(f"  测试集总体准确率: {metrics['overall_accuracy']:.4f}")
print(f"  测试集平均IoU: {metrics['mean_iou']:.4f}")
print(f"  测试集平均精确率: {metrics['mean_precision']:.4f}")
print(f"  测试集平均召回率: {metrics['mean_recall']:.4f}")

print(f"\n🎯 Vision Transformer特色:")
print(f"  ✓ 全局自注意力机制: 建立长程依赖关系")
print(f"  ✓ 位置编码: 提供空间位置信息")
print(f"  ✓ 多头注意力: 捕获不同类型的特征关系")
print(f"  ✓ 时空融合: 有效处理多光谱时序数据")

print(f"\n💾 保存的文件:")
save_dir = Path(config['save_dir'])
saved_files = list(save_dir.glob('*'))
for file in saved_files:
    print(f"  {file.name}")

print(f"\n🚀 使用建议:")
if RUNNING_MODE == "TEST":
    print(f"  ✅ 代码测试完成，可以切换到TRAIN模式进行完整训练")
    print(f"  📝 修改第一个cell: RUNNING_MODE = 'TRAIN'")
    print(f"  🔄 重新运行notebook进行完整训练")
    print(f"  ☁️ 建议在云端GPU上运行TRAIN模式")
else:
    print(f"  1. 模型已保存到: {config['save_dir']}/best_model.pth")
    print(f"  2. 用于推理: python -m Transformer.inference --model-path {config['save_dir']}/best_model.pth")
    print(f"  3. 注意力分析: 可视化注意力权重分布")
    print(f"  4. 模型部署: 考虑导出为ONNX格式")

print(f"\n📊 与其他模型对比:")
print(f"  vs TCN: 更强的全局建模能力")
print(f"  vs Swin Transformer: 更直接的全局注意力但计算复杂度更高")
print(f"  特点: 纯注意力机制，强大的特征学习能力")

print("\n" + "=" * 80)
if RUNNING_MODE == "TEST":
    print("🧪 Vision Transformer代码测试完成!")
    print("✅ 所有组件运行正常，可以进行完整训练!")
else:
    print("🎉 Vision Transformer农作物制图模型训练完成!")
    print("🌟 全局自注意力，强大特征学习，优秀泛化性能!")
print("=" * 80)

In [ ]:
print("=" * 80)
print("🎯 TRANSFORMER 农作物制图模型训练总结")
print("=" * 80)

print(f"\n📊 模型配置:")
print(f"  模型类型: Vision Transformer")
print(f"  嵌入维度: {config['embed_dim']}")
print(f"  层数: {config['num_layers']}")
print(f"  注意力头数: {config['num_heads']}")
print(f"  总参数量: {total_params:,}")

print(f"\n🏃 训练过程:")
print(f"  训练epochs: {len(history['train_loss'])}")
print(f"  总训练时间: {total_training_time/60:.1f}分钟")
print(f"  平均每epoch: {total_training_time/len(history['train_loss']):.1f}秒")
print(f"  最佳验证mIoU: {best_miou:.4f}")

print(f"\n📈 最终性能:")
print(f"  测试集总体准确率: {metrics['overall_accuracy']:.4f}")
print(f"  测试集平均IoU: {metrics['mean_iou']:.4f}")
print(f"  测试集平均精确率: {metrics['mean_precision']:.4f}")
print(f"  测试集平均召回率: {metrics['mean_recall']:.4f}")

print(f"\n💾 保存的文件:")
save_dir = Path(config['save_dir'])
saved_files = list(save_dir.glob('*'))
for file in saved_files:
    print(f"  {file.name}")

print(f"\n🚀 使用建议:")
print(f"  1. 模型已保存到: {config['save_dir']}/best_model.pth")
print(f"  2. 用于推理: python -m Transformer.inference --model-path {config['save_dir']}/best_model.pth")
print(f"  3. 继续训练: 加载checkpoint并调整学习率")
print(f"  4. 模型部署: 考虑量化或剪枝以减少计算需求")

print("\n" + "=" * 80)
print("🎉 Transformer农作物制图模型训练完成!")
print("=" * 80)